peewee ORM. Offline examples. Full docs here: http://docs.peewee-orm.com/en/latest/index.html

Setting up database

In [1]:
from peewee import *
from datetime import date

db = SqliteDatabase('people2.db')

class Person(Model):
    name = CharField()
    birthday = DateField()
    is_relative = BooleanField()

    class Meta:
        database = db
        
class Pet(Model):
    owner = ForeignKeyField(Person, related_name='pets') #relationship
    name = CharField()
    animal_type = CharField()

    class Meta:
        database = db

In [2]:
# Setting up

db.connect()
db.create_tables([Person, Pet])

In [3]:
# Creating

uncle_bob = Person(name='Bob', birthday=date(1960, 1, 15), is_relative=True)
uncle_bob.save() # bob is now stored in the database
# When you call save(), the number of rows modified is returned.

# Alternative way of creating new rows
grandma = Person.create(name='Grandma', birthday=date(1935, 3, 1), is_relative=True)
herb = Person.create(name='Herb', birthday=date(1950, 5, 5), is_relative=False)

# Creating pets assigned to people
bob_kitty = Pet.create(owner=uncle_bob, name='Kitty', animal_type='cat')
herb_fido = Pet.create(owner=herb, name='Fido', animal_type='dog')
herb_mittens = Pet.create(owner=herb, name='Mittens', animal_type='cat')
herb_mittens_jr = Pet.create(owner=herb, name='Mittens Jr', animal_type='cat')

In [4]:
# Updating
grandma.name = 'Grandma L.'
grandma.save()

1

In [5]:
# Deleting
herb_mittens.delete_instance()
# The return value of delete_instance() is the number of rows removed from the database.

1

In [6]:
# Select query
grandma = Person.select().where(Person.name == 'Grandma L.').get()
# Alternatively
grandma = Person.get(Person.name == 'Grandma L.')

In [7]:
# Select many records
for person in Person.select():
    print(person.name, person.is_relative)

Bob True
Grandma L. True
Herb False


In [8]:
# Where clause
# Not recommended, N+1 behavior, pet.owner.name creates additional query which is unwanted
query = Pet.select().where(Pet.animal_type == 'cat')
for pet in query:
    print(pet.name, pet.owner.name)

Kitty Bob
Mittens Jr Herb


In [9]:
# Proper where clause. Joined Pet and Person table
query = (Pet
          .select(Pet, Person)
          .join(Person)
          .where(Pet.animal_type == 'cat'))
for pet in query:
     print(pet.name, pet.owner.name)

Kitty Bob
Mittens Jr Herb


In [10]:
for pet in Pet.select().join(Person).where(Person.name == 'Bob'):
     print(pet.name)

Kitty


In [11]:
for person in Person.select().order_by(Person.birthday.desc()):
     print(person.name, person.birthday)

Bob 1960-01-15
Herb 1950-05-05
Grandma L. 1935-03-01


In [12]:
subquery = Pet.select(fn.COUNT(Pet.id)).where(Pet.owner == Person.id)
query = (Person
          .select(Person, Pet, subquery.alias('pet_count'))
          .join(Pet, JOIN.LEFT_OUTER)
          .order_by(Person.name))

for person in query.aggregate_rows():  # Note the `aggregate_rows()` call.
     print(person.name, person.pet_count, 'pets')
     for pet in person.pets:
         print('    ', pet.name, pet.animal_type)

Bob 1 pets
     Kitty cat
Grandma L. 0 pets
Herb 2 pets
     Fido dog
     Mittens Jr cat


In [13]:
d1940 = date(1940, 1, 1)
d1960 = date(1960, 1, 1)
query = (Person
          .select()
          .where((Person.birthday < d1940) | (Person.birthday > d1960)))

for person in query:
     print(person.name, person.birthday)

Bob 1960-01-15
Grandma L. 1935-03-01
